# Stegasus

## Commons

### Colab Specific

In [1]:
!apt-get install openjdk-17-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-17-openjdk-amd64/bin/java
!java -version

The system cannot find the path specified.
'update-alternatives' is not recognized as an internal or external command,
operable program or batch file.


openjdk version "17.0.7" 2023-04-18 LTS
OpenJDK Runtime Environment Corretto-17.0.7.7.1 (build 17.0.7+7-LTS)
OpenJDK 64-Bit Server VM Corretto-17.0.7.7.1 (build 17.0.7+7-LTS, mixed mode, sharing)


In [2]:
!git clone https://github.com/NasoohOlabi/Stegasus.git

^C


In [ ]:
!mv Stegasus/* .
!rm -r Stegasus

mv: cannot move 'Stegasus/Benchmarks' to './Benchmarks': Directory not empty
mv: cannot move 'Stegasus/Bot' to './Bot': Directory not empty
mv: cannot move 'Stegasus/Emojier' to './Emojier': Directory not empty
mv: cannot move 'Stegasus/FrustratinglySimpleBert' to './FrustratinglySimpleBert': Directory not empty
mv: cannot move 'Stegasus/openai' to './openai': Directory not empty
mv: cannot move 'Stegasus/PersonaGPTBot' to './PersonaGPTBot': Directory not empty
mv: cannot move 'Stegasus/SampleData' to './SampleData': Directory not empty
mv: cannot move 'Stegasus/SampleOutputs' to './SampleOutputs': Directory not empty
mv: cannot move 'Stegasus/SemanticMasking' to './SemanticMasking': Directory not empty
mv: cannot move 'Stegasus/Typoceros' to './Typoceros': Directory not empty


In [ ]:
%pip install -r requirements.txt

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl (12.8 MB)


In [ ]:
ROOT_DIR = '.'

### General Common

In [1]:
#@title random_bit_stream
import random

def random_bit_stream(length=None):
    """Return a random string of zeros and ones of the given length (default: random integer between 0 and 100)."""
    if length is None:
        length = random.randint(0, 100)
    return ''.join(str(random.randint(0, 1)) for _ in range(length))
def int_to_binary_string(n: int, length: int):
    binary_str = bin(n)[2:]  # convert to binary string, remove '0b' prefix
    padded_str = binary_str.rjust(length, '0')  # pad with zeros to length
    return padded_str

In [2]:
random_bit_stream(13)

'0111011010011'

In [3]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Stegasus Import

In [4]:
from stegasus import StegasusEncode, StegasusDecode

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nasoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Setting up Masked Stego



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Setting up Masked Stego Completed


Setting up Typoceros

('hi, how atre you?', '01001011111')

Setting up Typoceros Completed



In [5]:
StegasusEncode('hi, how are you?','10101010010101')

('hi, how atre you?', '01010010101')

In [6]:
# FIXME
StegasusDecode('hi, how atre you?')

('hi, how are you?', '101')

In [7]:
LONG_TEXT = '''Metaphysical solipsism is a variety of solipsism. Based on a philosophy of subjective idealism, metaphysical solipsists maintain that the self is the only existing reality and that all other realities, including the external world and other persons.'''

In [8]:
Famous_Demo = 'Hi, How are you?'

In [9]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = (end_time - start_time) / 60
        print(f"Function '{func.__name__}' took {elapsed_time:.2f} mins to execute.")
        return result
    return wrapper

In [10]:
# bench and test
@timer
def bench(xtx:str,EncodeOnly=False):
  # print('#'*40)
  data = random_bit_stream(len(xtx))
  # print(f'original={xtx}')
  encoded,rem = StegasusEncode(xtx,data)
  if not EncodeOnly:
    org,deData = StegasusDecode(encoded)
    print(f'data={data[:-len(rem)]}')
    print(f'encoded={encoded}')
    print(f'capacity={100*((len(xtx) - len(rem)) / len(xtx))}%\tdata length={len(data)-len(rem)}\ttext length={len(xtx)}')

In [11]:
from SampleData import ConversationsRepo

## Note: the bench time is twice the engine time since bench encode and decode

In [12]:
for j in range(5):
  print(f'j={j}')
  for i, m in enumerate(ConversationsRepo.get(39)):
    bench(m,True)
  break

j=0
Function 'bench' took 0.12 mins to execute.
Function 'bench' took 0.20 mins to execute.
Function 'bench' took 0.16 mins to execute.
Function 'bench' took 0.27 mins to execute.
Function 'bench' took 0.66 mins to execute.
Function 'bench' took 0.22 mins to execute.
Function 'bench' took 0.04 mins to execute.
Function 'bench' took 0.80 mins to execute.
Function 'bench' took 0.05 mins to execute.
Function 'bench' took 0.20 mins to execute.
Function 'bench' took 0.07 mins to execute.
Function 'bench' took 0.16 mins to execute.
Function 'bench' took 0.10 mins to execute.
Function 'bench' took 0.19 mins to execute.
Function 'bench' took 0.19 mins to execute.
Function 'bench' took 0.15 mins to execute.
Function 'bench' took 0.10 mins to execute.
Function 'bench' took 0.21 mins to execute.
Function 'bench' took 0.09 mins to execute.
Function 'bench' took 0.23 mins to execute.
Function 'bench' took 0.03 mins to execute.


## Benchmarks

In [13]:
from SampleData import ConversationsRepo
import random
from Bot import *

ModuleNotFoundError: No module named 'Bot'

In [ ]:
def test_stream(minimum=1,maximum=ConversationsRepo.ConversationsCount):
  gened = set()
  for i in range(maximum - minimum):
    x = random.randint(minimum,maximum)
    while x in gened:
      x = random.randint(minimum,maximum)
    gened.add(x)
    yield x

In [ ]:
# StegasusEncode("I sure do. You can listen to JUpiter's storms on AM radio",random_bit_stream(1000))

("I sure do 😃😃. You can liosten to JUpiter's storms 😋 on AM radio",
 '011110101110100000101000110100000111011011101010011110000011100110000110101000001101001011101011010100000011101110100111010000100110000010010011000011111000101001010100001101101100100011111011011000100100001010101000100111101011001010100000010111001000010111101110000011111111110111100000101100101000110010100101001110101101100111101001100100101001000011001001001111010001101001110111011010011111000011110000101011010000101111111001001100110101011101110010110000010101101111010000000101000101011110111001111010101110011000101100100000011001110101000010101101111101001010111000100010100111010010101010101000011100111100011001011111011110101101001011100110100011000001100010100110001100011010000011001100101111001111111001011101011001100111100011011011001101101110110100111010110101011101100110101000100111110001011001011111011110000001110100110100000000110011111110000011111011010111111101100101001000011000101110001111111001101010

In [ ]:
def r():
  for idx in test_stream():
    conversation = ConversationsRepo.get(idx)
    # chat = Chat(Person('Person','A',18,'male','Damascus','a@gmail.com',{},[]),Person('Person','B',18,'female','Damascus','b@gmail.com',{},[]),lambda x: x)
    people = [Person('Alice','A',18,'male','Damascus','a@gmail.com',{},[]),Person('Bob','B',18,'female','Damascus','b@gmail.com',{},[])]
    estimated_length = sum(map(len,conversation))
    messages = [Message(people[i%2],m) for i,m in enumerate(conversation)]
    chat = Chat(people[0],people[1],lambda x : x)
    chat.messages = messages
    print(f'idx={idx},payload size={estimated_length * 2}')
    chat.render()
    payload = random_bit_stream(estimated_length * 2)
    for idx in range(len(messages)):
      carrier_size = len(messages[idx].text)
      messages[idx].text, rem = StegasusEncode(messages[idx].text,payload)
      print(f'text length={carrier_size}\tencoded bits={len(payload)-len(rem)}\tratio={int(((len(payload)-len(rem))/carrier_size)*100)}%\ttext={messages[idx].text}')
      payload = rem
    chat.render()
# r()

idx=612,payload size=5098
--------------------- Chat ---------------------
Alice: ` Hi, do you want to chat about Mars? Mars is the fourth planet from the Sun and the second-smallest planet in the Solar System after Mercury. In English, Mars carries a name of the Roman God of War and is often referred to as the Red Planet.`
Bob: ` I would say that Mars is a fierce looking neighbor, but probably the friendliest planet at least for humans. Second to earth, that is.`
Alice: ` Yes, I agree, it's reddish iron oxide prevalent on its surface gives it a reddish appearance that is distinctive among the astronomical bodies visible to the naked eye.`
Bob: ` I think the red is clear from a distance, but up close, it's not quite so red. Plus I bet the atmosphere changes the appearance of a lot of things. And sound, too.`
Alice: ` Well, Mars is a terrestrial planet with a thin atmosphere having surfaces like the impact craters of the Moon and the valleys, deserts, and polar ice caps of Earth.`
Bob: 

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./stats2.tsv',delimiter='\t')

In [ ]:
df['bits'] = df['emo bits']+df['fsb bits']+df['typo bits']
df['cover size'] = 100 * df['bits'] / (df['emo capacity']+df['fsb capacity']+df['typo capacity'])
df['cover size'] = df['cover size'].apply(lambda x : int(round(x)))

df['capacity'] = 100 * df['bits'] / df['cover size']
df['capacity'] = df['capacity'].apply(lambda x : int(round(x)))

In [ ]:
df.to_csv('./stats2.tsv',sep='\t',index=False)